<a href="https://colab.research.google.com/github/justice-7/Data-Analysis-Programming/blob/main/DAP_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8(202012770_%EB%B0%95%EC%A0%95%EC%9D%98).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 주식 정보 크롤링
참고:
https://blog.naver.com/PostView.nhn?isHttpsRedirect=true&blogId=ym7596&logNo=222056342589&categoryNo=0&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView

In [ ]:
!pip install selenium
!pip install html_table_parse
!pip install pandas
!pip install beutifulsoup4 
!pip install --upgrade beautifulsoup4
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver

import pandas as pd
import time
import xlwt

import numpy as np  
import glob  
import sys
import os

from selenium.webdriver.support.ui import Select

options = webdriver.ChromeOptions()
options.add_argument('--headless')       
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

def get_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
        if len(prod_item) <= 1:
            continue
    
        try:
            name = prod_item.select('td')[1].text.strip()
        except:
            name = ''
        try:
            price = prod_item.select('td')[2].text.strip()
        except:
            price = '0'
        try:
             georae= prod_item.select('td')[6].text.strip()
        except:
            georae = '0'
        try:
            stockCnt = prod_item.select('td')[7].text.strip()
        except:
            stockCnt = '0'
        try:
            eps = prod_item.select('td')[8].text.strip()
        except:
            eps = '0'
        try:
            per = prod_item.select('td')[9].text.strip()
        except:
            per = '0'
        try:
            roe = prod_item.select('td')[10].text.strip()
        except:
            roe = '0'
        try:
            pbr = prod_item.select('td')[11].text.strip()
        except:
            pbr = '0'
        
   
        prod_data.append([name,price,georae,stockCnt,eps,per,roe,pbr])
    return prod_data

[os.remove(f) for f in glob.glob("./stock*.csv")] # 만들어진 파일 삭제(stock*.csv)

for i in range(1,7):
    url='https://finance.naver.com/sise/sise_market_sum.naver?&page='+str(i)
    driver.get(url)
    time.sleep(2)

    if(i==1):
        driver.find_element_by_id('option4').click()
        driver.find_element_by_id('option15').click()
        driver.find_element_by_id('option23').click()
        driver.find_element_by_id('option24').click()

        #원하는 항목을 체크 후 적용하기 클릭
        driver.find_element_by_xpath('/html/body/div[3]/div[2]/div[2]/div[1]/div[2]/form/div/div/div/a[1]/img').click()

    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    prod_items = soup.select('table.type_2 > tbody > tr')

    prod_data_fin = get_items(prod_items)

    data = pd.DataFrame(prod_data_fin)

    data.columns = ['이름','현재가','거래량','상장주식수','주당순이익','PER','ROE','PBR']

    data.to_csv('./stock'+str(i)+'.csv')

    time.sleep(2)

all_data = []  
for f in glob.glob('./stock*.csv'): # stock으로 시작하는 파일  
    df = pd.read_csv(f)  
    all_data.append(df)

dataCombine = pd.concat(all_data, axis=0, ignore_index=True)
# 파일저장 
dataCombine.to_csv("./stock.csv", index=False)



In [86]:
df=pd.read_csv('./stock.csv')
df=df[['이름','현재가','거래량','상장주식수','주당순이익','PER','ROE','PBR']]
df

,이름,현재가,거래량,상장주식수,주당순이익,PER,ROE,PBR
0,KODEX 단기채권,"102,990","43,483","18,871",NaN,NaN,NaN,NaN
1,효성,"91,100","48,646","21,071","18,374",4.96,-2.26,0.71
2,KODEX 레버리지,"21,650","14,392,796","83,600",NaN,NaN,NaN,NaN
3,대웅,"30,600","62,670","58,142","1,816",16.85,15.1,1.82
4,신풍제약,"33,500","2,442,138","52,985",6,"5,583.33",1.72,4.70
...,...,...,...,...,...,...,...,...
295,하이트진로,"29,700","160,438","70,134",734,40.46,8.23,1.90
296,현대해상,"23,150","399,404","89,400","4,320",5.36,7.13,0.39
297,롯데정밀화학,"78,900","84,516","25,800","16,517",4.78,12.65,1.02
298,코오롱인더,"71,400","165,582","27,519","6,443",11.08,9.7,0.88
